In [28]:
import pandas as pd
import geopandas as gpd
from data_processing import enc_cocina, enc_distrito, enc_barrio, mlb
import ast

In [29]:
print(mlb.classes_)

['amusement_park' 'bakery' 'bar' 'cafe' 'casino' 'convenience_store'
 'grocery_or_supermarket' 'gym' 'health' 'liquor_store' 'lodging'
 'meal_delivery' 'meal_takeaway' 'movie_theater' 'night_club' 'parking'
 'real_estate_agency' 'spa' 'stadium' 'store' 'supermarket'
 'tourist_attraction' 'travel_agency']


In [30]:
data_usuario = {
        "price_level": [1],
        "tipo_cocina":['Española'],
        "serves_vegetarian_food":[0],
        "serves_wine":[1],
        "serves_beer":[1],
        "dine_in":[1],
        "reservable":[1],
        "takeout":[1],
        "delivery":[1],
        "weelchair":[1],
        "serves_breakfast": [0],
        "serves_brunch": [0],
        "serves_lunch": [1],
        "serves_dinner": [1],
        "open_weekends": [1],
        "hours_open": [80],
        "num_days_open": [7],
        'cod_distrito': ['1'],
        'cod_barrio': ['011'],
        'types':["['bar']"]
    }

df_usuario = pd.DataFrame(data_usuario)

In [31]:
barrio = gpd.read_file('../data/raw/Barrios.json') # lectura
barrio = gpd.GeoDataFrame(barrio, geometry='geometry', crs='EPSG:4326')
barrio_elegido = barrio[barrio['COD_BAR']==df_usuario['cod_barrio'].values[0]]

In [32]:
barrio_elegido = barrio_elegido.to_crs(epsg=25830)
barrio_elegido['centroid'] = barrio_elegido.geometry.centroid # calculamos centroides de barrios
centroides_latlon = barrio_elegido.set_geometry('centroid').to_crs(epsg=4326)
barrio_elegido['lon'] = centroides_latlon.geometry.x
barrio_elegido['lat'] = centroides_latlon.geometry.y
barrio_elegido['centroid'] = centroides_latlon.geometry

In [33]:
barrio_elegido = barrio_elegido[['CODDIS', 'NOMDIS', 'COD_BAR', 'NOMBRE','lon', 'lat', 'centroid']]

# Para cada cada centroide calculamos un buffer de 500 para mirar que restaurantes tenemos cerca
barrios_geo = gpd.GeoDataFrame(barrio_elegido, geometry='centroid', crs='EPSG:4326')
barrios_geo = barrios_geo.to_crs(epsg=25830)
barrios_geo['buffer_500'] = barrios_geo.centroid.buffer(500) #creamos el campo de buffer
barrios_geo = barrios_geo.set_geometry('buffer_500')
# leemos los restaurantes de madrid
restaurantes = pd.read_csv('../data/processed/restaurantes.csv')
restaurantes = restaurantes[['lat', 'lon', 'place_id', 'price_level', 'rating', 
                            'user_ratings_total', 'anio_medio_constr_vivendas',
                            'dur_media_credito_viviendas', 'edad_media_poblacion',
                            'num_locales_alta_abiertos', 'num_locales_alta_cerrados',
                            'poblacion_densidad', 'renta_media_persona',
                            'pct_crecimiento_demografico', 'valor_catast_inmueble_residen',
                            'tasa_parados', 'poblacion_80_mas',
                            'poblacion_italia', 'poblacion_china',
                            'cod_barrio']]

# creamos un df de los kpi por barrio
kpi = restaurantes.groupby('cod_barrio')[['anio_medio_constr_vivendas',
                                        'dur_media_credito_viviendas',
                                        'edad_media_poblacion',
                                        'num_locales_alta_abiertos',
                                        'num_locales_alta_cerrados',
                                        'poblacion_densidad',
                                        'renta_media_persona',
                                        'pct_crecimiento_demografico',
                                        'valor_catast_inmueble_residen',
                                        'tasa_parados',
                                        'poblacion_80_mas',
                                        'poblacion_italia',
                                        'poblacion_china']].max().reset_index()

restaurantes_geo = gpd.GeoDataFrame(restaurantes, geometry=gpd.points_from_xy(restaurantes['lon'], restaurantes['lat']), crs='EPSG:4326')
restaurantes_geo = restaurantes_geo.to_crs(epsg=25830)

result_restaurantes = gpd.sjoin(restaurantes_geo, barrios_geo, how='right', predicate='intersects')
result = result_restaurantes.groupby(['COD_BAR'])[['price_level', 'rating', 'user_ratings_total']].mean().reset_index()
result2 = result_restaurantes.groupby(['COD_BAR'])[['place_id']].count().reset_index()
result = pd.merge(result, result2, left_on=['COD_BAR'], right_on=['COD_BAR'])
result.rename({
            'price_level':'price_level_mean',
            'rating':'rating_mean',
            'user_ratings_total':'user_ratings_mean',
            'place_id':'num_restaurantes'}, axis=1, inplace=True)

# Resultado de buffer
barrio_elegido = pd.merge(barrio_elegido, result, how='left', left_on='COD_BAR', right_on='COD_BAR')
restaurantes.fillna(0, inplace=True)

# Convertimos a int
barrio_elegido['COD_BAR'] = barrio_elegido['COD_BAR'].astype('int')
barrio_elegido['CODDIS'] = barrio_elegido['CODDIS'].astype('int')

# Incluir los kpi a barrios
barrio_elegido = pd.merge(barrio_elegido, kpi, how='left', left_on='COD_BAR', right_on='cod_barrio')

In [34]:
# Incluimos datos del usuario
for col in df_usuario.columns:
    barrio_elegido[col] = df_usuario.iloc[0][col]

# One hot enconder
tip_coci= enc_cocina.transform(barrio_elegido[['tipo_cocina']]).toarray()
tip_cocina_dummy = pd.DataFrame(tip_coci, columns=[cat for cat in enc_cocina.categories_[0]])

tip_distrito= enc_distrito.transform(barrio_elegido[['NOMDIS']]).toarray()
tip_distrito_dummy = pd.DataFrame(tip_distrito, columns=[cat for cat in enc_distrito.categories_[0]])

tip_barrio= enc_barrio.transform(barrio_elegido[['NOMBRE']]).toarray()
tip_barrio_dummy = pd.DataFrame(tip_barrio, columns=[cat for cat in enc_barrio.categories_[0]])

barrio_elegido = pd.concat([barrio_elegido, tip_cocina_dummy, tip_distrito_dummy, tip_barrio_dummy], axis=1)

# MiltilabelBinarizer

barrio_elegido['types'] = [ast.literal_eval(x) for x in barrio_elegido['types']]

types_encode = mlb.transform(barrio_elegido['types'])
types_df = pd.DataFrame(types_encode, columns=mlb.classes_)

barrio_elegido = pd.concat([barrio_elegido, types_df], axis=1) 


In [35]:
barrio_elegido

,CODDIS,NOMDIS,COD_BAR,NOMBRE,lon,lat,centroid,price_level_mean,rating_mean,user_ratings_mean,...,movie_theater,night_club,parking,real_estate_agency,spa,stadium,store,supermarket,tourist_attraction,travel_agency
0,1,Centro,11,Palacio,-3.714071,40.415428,POINT (-3.71407 40.41543),1.83871,4.241935,2083.322581,...,0,0,0,0,0,0,0,0,0,0


In [36]:

# Quitamos Columnas
barrio_elegido.drop(['NOMDIS', 'NOMBRE', 'centroid', 'cod_barrio', 'tipo_cocina'], axis=1, inplace=True)

barrio_elegido.rename({'CODDIS':'cod_distrito',
                'COD_BAR':'cod_barrio'}, inplace=True, axis=1)

# Ordenamos columnas para el modelo
barrio_elegido = barrio_elegido[['lat','lon','dine_in','price_level','reservable','serves_beer',
                                'serves_breakfast','serves_brunch','serves_dinner','serves_lunch',
                                'serves_vegetarian_food','serves_wine','takeout','delivery',
                                'weelchair','hours_open','num_days_open','open_weekends','amusement_park',
                                'bakery','bar','cafe','casino','convenience_store','grocery_or_supermarket',
                                'gym','health','liquor_store','lodging','meal_delivery','meal_takeaway',
                                'movie_theater','night_club','parking','real_estate_agency','spa',
                                'stadium','store','supermarket','tourist_attraction','travel_agency',
                                'price_level_mean','rating_mean','user_ratings_mean','num_restaurantes',
                                'anio_medio_constr_vivendas','dur_media_credito_viviendas','edad_media_poblacion',
                                'num_locales_alta_abiertos','num_locales_alta_cerrados','poblacion_densidad',
                                'renta_media_persona','pct_crecimiento_demografico','valor_catast_inmueble_residen',
                                'tasa_parados','poblacion_80_mas','poblacion_italia','poblacion_china','Americana / Burgers',
                                'Asiática','China','Española','Fusión','Italiana','Japonesa','Latinoamericana','Mexicana',
                                'Otros','Arganzuela','Carabanchel','Centro','Chamartín','Chamberí','Ciudad Lineal','Fuencarral - El Pardo',
                                'Hortaleza','Latina','Moncloa - Aravaca','Moratalaz','Puente de Vallecas','Retiro','Salamanca',
                                'Tetuán','Usera','Abrantes','Acacias','Adelfas','Almagro','Almenara','Almendrales','Aluche',
                                'Apóstol Santiago','Arapiles','Argüelles','Atalaya','Atocha','Bellas Vistas','Berruguete',
                                'Canillas','Casa de Campo','Castellana','Castilla','Castillejos','Chopera','Ciudad Jardín',
                                'Ciudad Universitaria','Colina','Comillas','Cortes','Costillares','Cuatro Caminos','Delicias',
                                'El Pardo','El Viso','Embajadores','Entrevías','Estrella','Fontarrón','Fuente del Berro',
                                'Fuentelarreina','Gaztambide','Goya','Guindalera','Hispanoamérica','Ibiza','Imperial',
                                'Justicia','La Concepción','La Paz','Legazpi','Lista','Los Cármenes','Los Jerónimos','Lucero',
                                'Marroquina','Media Legua','Mirasierra','Moscardó','Niño Jesús','Nueva España','Numancia',
                                'Opañel','Pacífico','Palacio','Palomeras Bajas','Palos de la Frontera','Peñagrande','Pilar',
                                'Pinar del Rey','Piovera','Portazgo','Pradolongo','Prosperidad','Puerta Bonita','Puerta del Ángel',
                                'Quintana','Recoletos','Ríos Rosas','San Diego','San Isidro','San Juan Bautista','San Pascual',
                                'Sol','Trafalgar','Universidad','Valdeacederas','Valdefuentes','Valdezarza','Vallehermoso',
                                'Valverde','Ventas','Vista Alegre','Zofío']]

In [39]:
barrio_elegido[barrio_elegido.isna().any(axis=1)]

,lat,lon,dine_in,price_level,reservable,serves_beer,serves_breakfast,serves_brunch,serves_dinner,serves_lunch,...,Trafalgar,Universidad,Valdeacederas,Valdefuentes,Valdezarza,Vallehermoso,Valverde,Ventas,Vista Alegre,Zofío


In [176]:
data_usuario = {
        "price_level": [1],
        "tipo_cocina":['Española'],
        "serves_vegetarian_food":[0],
        "serves_wine":[1],
        "serves_beer":[1],
        "dine_in":[1],
        "reservable":[1],
        "takeout":[1],
        "delivery":[1],
        "weelchair":[1],
        "serves_breakfast": [0],
        "serves_brunch": [0],
        "serves_lunch": [1],
        "serves_dinner": [1],
        "open_weekends": [1],
        "hours_open": [80],
        "num_days_open": [7],
        'cod_distrito': ['1'],
        'cod_barrio': ['011'],
        'types':["['bar']"]
    }

df_user = pd.DataFrame(data_usuario)

In [177]:
barrios_m30 = [
        "011", "012", "013", "014", "015", "016",
        "021", "022", "023", "024", "025", "026", 
        "031", "032", "033", "034", "035", "036",
        "041", "042", "043", "044", "045", "046",
        "051", "052", "053", "074", "075","076",
        "061", "062", "063", "064", "065", "066",
        "071", "072", "073", "092", "054", "055",
        "081", "091", "121", "056", "085", "027",
        "131", "132", "133", "151", "093", "094",
        "084"
    ]

In [178]:
barrios_mapa = gpd.read_file('../data/raw/Barrios.json') # lectura
barrios_mapa = gpd.GeoDataFrame(barrio, geometry='geometry', crs='EPSG:4326')

In [179]:
barrios = barrios_mapa[barrios_mapa['COD_BAR'].isin(barrios_m30)]

# Calcular centroides
barrios_proj = barrios.to_crs(epsg=25830)
barrios_proj['centroid'] = barrios_proj.geometry.centroid
centroides_latlon = barrios_proj.set_geometry('centroid').to_crs(epsg=4326)
barrios['lon'] = centroides_latlon.geometry.x
barrios['lat'] = centroides_latlon.geometry.y
barrios['centroid'] = centroides_latlon.geometry
barrios = barrios[['CODDIS', 'NOMDIS', 'COD_BAR', 'NOMBRE', 'lon', 'lat', 'centroid']]

# Restaurantes y buffers
barrios_geo = gpd.GeoDataFrame(barrios, geometry='centroid', crs='EPSG:4326').to_crs(epsg=25830)
barrios_geo['buffer_500'] = barrios_geo.centroid.buffer(500)
barrios_geo = barrios_geo.set_geometry('buffer_500')

restaurantes = pd.read_csv('../data/processed/restaurantes.csv')
restaurantes = restaurantes[['lat', 'lon', 'place_id', 'price_level', 'rating',
                                'user_ratings_total', 'anio_medio_constr_vivendas',
                                'dur_media_credito_viviendas', 'edad_media_poblacion',
                                'num_locales_alta_abiertos', 'num_locales_alta_cerrados',
                                'poblacion_densidad', 'renta_media_persona',
                                'pct_crecimiento_demografico', 'valor_catast_inmueble_residen',
                                'tasa_parados', 'poblacion_80_mas', 'poblacion_italia',
                                'poblacion_china', 'cod_barrio']]

kpi = restaurantes.groupby('cod_barrio').max().reset_index()
kpi.drop(['lat', 'lon', 'place_id', 'price_level', 'rating',
                                'user_ratings_total'], axis=1, inplace=True)

c:\Users\karli\AppData\Local\Programs\Python\Python313\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\karli\AppData\Local\Programs\Python\Python313\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\karli\AppData\Local\Programs\Python\Python313\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is tryin

In [180]:
restaurantes_geo = gpd.GeoDataFrame(
    restaurantes,
    geometry=gpd.points_from_xy(restaurantes['lon'], restaurantes['lat']),
    crs='EPSG:4326'
).to_crs(epsg=25830)

result_restaurantes = gpd.sjoin(restaurantes_geo, barrios_geo, how='right', predicate='intersects')
result = result_restaurantes.groupby(['COD_BAR'])[['price_level', 'rating', 'user_ratings_total']].mean().reset_index()
result2 = result_restaurantes.groupby(['COD_BAR'])[['place_id']].count().reset_index()
result = pd.merge(result, result2, on='COD_BAR')
result.rename(columns={
    'price_level': 'price_level_mean',
    'rating': 'rating_mean',
    'user_ratings_total': 'user_ratings_mean',
    'place_id': 'num_restaurantes'
}, inplace=True)

barrios = pd.merge(barrios, result, how='left', on='COD_BAR')
restaurantes.fillna(0, inplace=True)

barrios['COD_BAR'] = barrios['COD_BAR'].astype(int)
barrios['CODDIS'] = barrios['CODDIS'].astype(int)

In [182]:
barrios = pd.merge(barrios, kpi, how='left', left_on='COD_BAR', right_on='cod_barrio')

# Añadir valores del usuario
for col in df_user.columns:
    valor = df_user.iloc[0][col]
    barrios[col] = [valor] * len(barrios) if isinstance(valor, list) else valor

In [183]:
barrios.head(1)

,CODDIS,NOMDIS,COD_BAR,NOMBRE,lon,lat,centroid,price_level_mean,rating_mean,user_ratings_mean,...,weelchair,serves_breakfast,serves_brunch,serves_lunch,serves_dinner,open_weekends,hours_open,num_days_open,cod_distrito,types
0,1,Centro,11,Palacio,-3.714071,40.415428,POINT (-3.71407 40.41543),1.83871,4.241935,2083.322581,...,1,0,0,1,1,1,80,7,1,['bar']


In [184]:
# ---- One Hot Encoding protegido ----
if 'tipo_cocina' in barrios.columns and not barrios['tipo_cocina'].isnull().any():
    tip_coci = enc_cocina.transform(barrios[['tipo_cocina']]).toarray()
    tip_cocina_dummy = pd.DataFrame(tip_coci, columns=enc_cocina.categories_[0])
else:
    tip_cocina_dummy = pd.DataFrame(0, index=barrios.index, columns=enc_cocina.categories_[0])

if 'NOMDIS' in barrios.columns and not barrios['NOMDIS'].isnull().any():
    tip_distrito = enc_distrito.transform(barrios[['NOMDIS']]).toarray()
    tip_distrito_dummy = pd.DataFrame(tip_distrito, columns=enc_distrito.categories_[0])
else:
    tip_distrito_dummy = pd.DataFrame(0, index=barrios.index, columns=enc_distrito.categories_[0])

if 'NOMBRE' in barrios.columns and not barrios['NOMBRE'].isnull().any():
    tip_barrio = enc_barrio.transform(barrios[['NOMBRE']]).toarray()
    tip_barrio_dummy = pd.DataFrame(tip_barrio, columns=enc_barrio.categories_[0])
else:
    tip_barrio_dummy = pd.DataFrame(0, index=barrios.index, columns=enc_barrio.categories_[0])

barrios = pd.concat([barrios, tip_cocina_dummy, tip_distrito_dummy, tip_barrio_dummy], axis=1)

In [185]:
# ---- MultiLabelBinarizer protegido ----
def convertir_a_lista(x):
    if isinstance(x, list):
        return x
    try:
        return ast.literal_eval(x)
    except Exception:
        return []

def filtrar_etiquetas_validas(lista, clases_validas):
    lista_filtrada = []
    for sublista in lista:
        if isinstance(sublista, list):
            lista_filtrada.append([et for et in sublista if et in clases_validas])
        else:
            lista_filtrada.append([])
    return lista_filtrada

barrios['types'] = barrios['types'].apply(convertir_a_lista)
barrios['types'] = filtrar_etiquetas_validas(barrios['types'], mlb.classes_)

if barrios['types'].apply(lambda x: isinstance(x, list) and len(x) > 0).any():
    types_encoded = mlb.transform(barrios['types'])
    types_df = pd.DataFrame(types_encoded, columns=mlb.classes_)
else:
    types_df = pd.DataFrame(0, index=barrios.index, columns=mlb.classes_)

barrios = pd.concat([barrios, types_df], axis=1)

In [186]:
# ---- Limpieza final ----
barrios.drop(['NOMDIS', 'NOMBRE', 'centroid', 'cod_barrio', 'tipo_cocina'], axis=1, inplace=True)
barrios.rename(columns={'CODDIS': 'cod_distrito', 'COD_BAR': 'cod_barrio'}, inplace=True)


In [188]:
# Ordenamos columnas para el modelo
barrios = barrios[['lat','lon','dine_in','price_level','reservable','serves_beer',
                            'serves_breakfast','serves_brunch','serves_dinner','serves_lunch',
                            'serves_vegetarian_food','serves_wine','takeout','delivery',
                            'weelchair','hours_open','num_days_open','open_weekends','amusement_park',
                            'bakery','bar','cafe','casino','convenience_store','grocery_or_supermarket',
                            'gym','health','liquor_store','lodging','meal_delivery','meal_takeaway',
                            'movie_theater','night_club','parking','real_estate_agency','spa',
                            'stadium','store','supermarket','tourist_attraction','travel_agency',
                            'price_level_mean','rating_mean','user_ratings_mean','num_restaurantes',
                            'anio_medio_constr_vivendas','dur_media_credito_viviendas','edad_media_poblacion',
                            'num_locales_alta_abiertos','num_locales_alta_cerrados','poblacion_densidad',
                            'renta_media_persona','pct_crecimiento_demografico','valor_catast_inmueble_residen',
                            'tasa_parados','poblacion_80_mas','poblacion_italia','poblacion_china','Americana / Burgers',
                            'Asiática','China','Española','Fusión','Italiana','Japonesa','Latinoamericana','Mexicana',
                            'Otros','Arganzuela','Carabanchel','Centro','Chamartín','Chamberí','Ciudad Lineal','Fuencarral - El Pardo',
                            'Hortaleza','Latina','Moncloa - Aravaca','Moratalaz','Puente de Vallecas','Retiro','Salamanca',
                            'Tetuán','Usera','Abrantes','Acacias','Adelfas','Almagro','Almenara','Almendrales','Aluche',
                            'Apóstol Santiago','Arapiles','Argüelles','Atalaya','Atocha','Bellas Vistas','Berruguete',
                            'Canillas','Casa de Campo','Castellana','Castilla','Castillejos','Chopera','Ciudad Jardín',
                            'Ciudad Universitaria','Colina','Comillas','Cortes','Costillares','Cuatro Caminos','Delicias',
                            'El Pardo','El Viso','Embajadores','Entrevías','Estrella','Fontarrón','Fuente del Berro',
                            'Fuentelarreina','Gaztambide','Goya','Guindalera','Hispanoamérica','Ibiza','Imperial',
                            'Justicia','La Concepción','La Paz','Legazpi','Lista','Los Cármenes','Los Jerónimos','Lucero',
                            'Marroquina','Media Legua','Mirasierra','Moscardó','Niño Jesús','Nueva España','Numancia',
                            'Opañel','Pacífico','Palacio','Palomeras Bajas','Palos de la Frontera','Peñagrande','Pilar',
                            'Pinar del Rey','Piovera','Portazgo','Pradolongo','Prosperidad','Puerta Bonita','Puerta del Ángel',
                            'Quintana','Recoletos','Ríos Rosas','San Diego','San Isidro','San Juan Bautista','San Pascual',
                            'Sol','Trafalgar','Universidad','Valdeacederas','Valdefuentes','Valdezarza','Vallehermoso',
                            'Valverde','Ventas','Vista Alegre','Zofío']]